In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import collections

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Multi-Layer Perceptron (MLP)

## Softmax regression: No-hidden layer perceptron (WIP)

## 1-hidden layer percentron

In [3]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

w1 = tf.get_variable('w1', [784, 256], initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1))
b1 = tf.get_variable('b1', [256], initializer=tf.zeros_initializer)
w2 = tf.get_variable('w2', [256, 10], initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1))
b2 = tf.get_variable('b2', [10], initializer=tf.zeros_initializer)

h1 = tf.nn.sigmoid(tf.matmul(X, w1) + b1)
logits = tf.matmul(h1, w2) + b2
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [35]:
def train(epoch_n=20, batch_size=100, n_iter=None, use_training_ph=False):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    N = mnist.train.num_examples
    if n_iter is None:
        n_iter = N // batch_size
    dq = collections.deque(maxlen=5)
        
    for epoch in range(epoch_n):
        avg_loss = 0.
        avg_acc = 0.
        for _ in range(n_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_x, y: batch_y}
            if use_training_ph:
                feed_dict[training] = True
            _, cur_acc, cur_loss = sess.run([train_op, accuracy, loss], feed_dict=feed_dict)
            avg_acc += cur_acc
            avg_loss += cur_loss

        avg_acc /= n_iter
        avg_loss /= n_iter

        test_acc = 0.
        test_loss = 0.
        for _ in range(mnist.test.num_examples // 1000):
            batch_x, batch_y = mnist.test.next_batch(1000)
            feed_dict = {X: batch_x, y: batch_y}
            if use_training_ph:
                feed_dict[training] = False
            cur_acc, cur_loss = sess.run([accuracy, loss], feed_dict=feed_dict)
            test_acc += cur_acc
            test_loss += cur_loss
        test_acc /= (mnist.test.num_examples // 1000)
        test_loss /= (mnist.test.num_examples // 1000)
        
        print("[{:2}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
              format(epoch+1, epoch_n, avg_acc, avg_loss, test_acc, test_loss))
        dq.append(test_acc)
    
    score = np.average(dq)
    print("average of last 5 test acc: {:.2%}".format(score))

In [5]:
train()

[ 1/20] (train) acc: 80.62%, loss: 0.783 | (test) acc: 88.80%, loss: 0.426
[ 2/20] (train) acc: 88.88%, loss: 0.400 | (test) acc: 90.40%, loss: 0.348
[ 3/20] (train) acc: 90.13%, loss: 0.346 | (test) acc: 91.12%, loss: 0.312
[ 4/20] (train) acc: 90.76%, loss: 0.320 | (test) acc: 91.48%, loss: 0.294
[ 5/20] (train) acc: 91.30%, loss: 0.303 | (test) acc: 91.91%, loss: 0.284
[ 6/20] (train) acc: 91.66%, loss: 0.290 | (test) acc: 92.23%, loss: 0.272
[ 7/20] (train) acc: 91.98%, loss: 0.278 | (test) acc: 92.45%, loss: 0.262
[ 8/20] (train) acc: 92.30%, loss: 0.267 | (test) acc: 92.54%, loss: 0.256
[ 9/20] (train) acc: 92.70%, loss: 0.257 | (test) acc: 93.06%, loss: 0.247
[10/20] (train) acc: 92.93%, loss: 0.248 | (test) acc: 92.84%, loss: 0.241
[11/20] (train) acc: 93.22%, loss: 0.240 | (test) acc: 93.34%, loss: 0.231
[12/20] (train) acc: 93.47%, loss: 0.231 | (test) acc: 93.46%, loss: 0.230
[13/20] (train) acc: 93.64%, loss: 0.224 | (test) acc: 93.83%, loss: 0.217
[14/20] (train) acc: 93.9

## tf.slim

In [8]:
slim = tf.contrib.slim

In [7]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

h1 = slim.fully_connected(X, 256, activation_fn=tf.nn.sigmoid, 
                          weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1), 
                          biases_initializer=tf.zeros_initializer())
logits = slim.fully_connected(h1, 10, activation_fn=None, 
                              weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1), 
                              biases_initializer=tf.zeros_initializer())
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [8]:
train()

[ 1/20] (train) acc: 80.32%, loss: 0.787 | (test) acc: 88.90%, loss: 0.432
[ 2/20] (train) acc: 88.85%, loss: 0.401 | (test) acc: 90.55%, loss: 0.345
[ 3/20] (train) acc: 90.11%, loss: 0.347 | (test) acc: 90.92%, loss: 0.316
[ 4/20] (train) acc: 90.80%, loss: 0.320 | (test) acc: 91.68%, loss: 0.293
[ 5/20] (train) acc: 91.32%, loss: 0.302 | (test) acc: 91.79%, loss: 0.281
[ 6/20] (train) acc: 91.69%, loss: 0.288 | (test) acc: 92.05%, loss: 0.273
[ 7/20] (train) acc: 92.08%, loss: 0.276 | (test) acc: 92.54%, loss: 0.260
[ 8/20] (train) acc: 92.40%, loss: 0.265 | (test) acc: 92.73%, loss: 0.254
[ 9/20] (train) acc: 92.69%, loss: 0.255 | (test) acc: 93.07%, loss: 0.242
[10/20] (train) acc: 92.94%, loss: 0.246 | (test) acc: 93.12%, loss: 0.237
[11/20] (train) acc: 93.28%, loss: 0.237 | (test) acc: 93.41%, loss: 0.229
[12/20] (train) acc: 93.52%, loss: 0.228 | (test) acc: 93.64%, loss: 0.222
[13/20] (train) acc: 93.75%, loss: 0.220 | (test) acc: 93.88%, loss: 0.214
[14/20] (train) acc: 94.0

## Better MLP - ReLU, xavier_init, adam

In [9]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

h1 = slim.fully_connected(X, 256, activation_fn=tf.nn.relu, weights_initializer=slim.xavier_initializer())
logits = slim.fully_connected(h1, 10, activation_fn=None, weights_initializer=slim.xavier_initializer())
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [10]:
train()

[ 1/20] (train) acc: 91.67%, loss: 0.297 | (test) acc: 95.60%, loss: 0.150
[ 2/20] (train) acc: 96.31%, loss: 0.129 | (test) acc: 96.57%, loss: 0.109
[ 3/20] (train) acc: 97.41%, loss: 0.088 | (test) acc: 97.38%, loss: 0.085
[ 4/20] (train) acc: 98.14%, loss: 0.063 | (test) acc: 97.38%, loss: 0.081
[ 5/20] (train) acc: 98.55%, loss: 0.049 | (test) acc: 97.45%, loss: 0.079
[ 6/20] (train) acc: 98.91%, loss: 0.038 | (test) acc: 97.71%, loss: 0.070
[ 7/20] (train) acc: 99.20%, loss: 0.029 | (test) acc: 97.64%, loss: 0.073
[ 8/20] (train) acc: 99.36%, loss: 0.023 | (test) acc: 97.86%, loss: 0.065
[ 9/20] (train) acc: 99.54%, loss: 0.018 | (test) acc: 97.86%, loss: 0.068
[10/20] (train) acc: 99.64%, loss: 0.015 | (test) acc: 97.91%, loss: 0.069
[11/20] (train) acc: 99.75%, loss: 0.011 | (test) acc: 98.04%, loss: 0.063
[12/20] (train) acc: 99.82%, loss: 0.009 | (test) acc: 98.02%, loss: 0.071
[13/20] (train) acc: 99.77%, loss: 0.009 | (test) acc: 98.06%, loss: 0.070
[14/20] (train) acc: 99.8

## Going Deeper

In [11]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

h1 = slim.fully_connected(X, 256, activation_fn=tf.nn.relu, weights_initializer=slim.xavier_initializer())
h2 = slim.fully_connected(h1, 128, activation_fn=tf.nn.relu, weights_initializer=slim.xavier_initializer())
logits = slim.fully_connected(h2, 10, activation_fn=None, weights_initializer=slim.xavier_initializer())
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [12]:
train()

[ 1/20] (train) acc: 92.19%, loss: 0.268 | (test) acc: 96.13%, loss: 0.124
[ 2/20] (train) acc: 96.88%, loss: 0.103 | (test) acc: 96.89%, loss: 0.099
[ 3/20] (train) acc: 97.93%, loss: 0.067 | (test) acc: 97.53%, loss: 0.076
[ 4/20] (train) acc: 98.44%, loss: 0.049 | (test) acc: 97.91%, loss: 0.068
[ 5/20] (train) acc: 98.92%, loss: 0.034 | (test) acc: 97.87%, loss: 0.070
[ 6/20] (train) acc: 99.16%, loss: 0.027 | (test) acc: 97.83%, loss: 0.072
[ 7/20] (train) acc: 99.29%, loss: 0.022 | (test) acc: 97.73%, loss: 0.074
[ 8/20] (train) acc: 99.39%, loss: 0.018 | (test) acc: 97.91%, loss: 0.076
[ 9/20] (train) acc: 99.44%, loss: 0.016 | (test) acc: 98.17%, loss: 0.072
[10/20] (train) acc: 99.51%, loss: 0.015 | (test) acc: 98.03%, loss: 0.078
[11/20] (train) acc: 99.62%, loss: 0.012 | (test) acc: 98.05%, loss: 0.076
[12/20] (train) acc: 99.57%, loss: 0.013 | (test) acc: 98.19%, loss: 0.072
[13/20] (train) acc: 99.62%, loss: 0.011 | (test) acc: 98.11%, loss: 0.076
[14/20] (train) acc: 99.7

* No better acc ...

# Convolutional Neural Networks (CNN)

In [13]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(X, [-1, 28, 28, 1])

h1 = slim.conv2d(x_img, 32, kernel_size=[3,3]) # default relu, xavier
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
assert p1.shape[1:] == [14, 14, 32]

h2 = slim.conv2d(p1, 64, kernel_size=[3,3])
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
assert p2.shape[1:] == [7, 7, 64]

flat = slim.flatten(p2)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [14]:
train()

[ 1/20] (train) acc: 92.70%, loss: 0.250 | (test) acc: 97.81%, loss: 0.071
[ 2/20] (train) acc: 97.97%, loss: 0.066 | (test) acc: 98.68%, loss: 0.044
[ 3/20] (train) acc: 98.48%, loss: 0.047 | (test) acc: 98.84%, loss: 0.034
[ 4/20] (train) acc: 98.82%, loss: 0.038 | (test) acc: 98.85%, loss: 0.034
[ 5/20] (train) acc: 98.95%, loss: 0.032 | (test) acc: 98.79%, loss: 0.037
[ 6/20] (train) acc: 99.13%, loss: 0.027 | (test) acc: 98.87%, loss: 0.033
[ 7/20] (train) acc: 99.26%, loss: 0.022 | (test) acc: 98.86%, loss: 0.033
[ 8/20] (train) acc: 99.36%, loss: 0.020 | (test) acc: 98.87%, loss: 0.036
[ 9/20] (train) acc: 99.46%, loss: 0.017 | (test) acc: 98.88%, loss: 0.035
[10/20] (train) acc: 99.55%, loss: 0.014 | (test) acc: 98.56%, loss: 0.044
[11/20] (train) acc: 99.59%, loss: 0.013 | (test) acc: 98.99%, loss: 0.032
[12/20] (train) acc: 99.67%, loss: 0.011 | (test) acc: 98.85%, loss: 0.038
[13/20] (train) acc: 99.75%, loss: 0.008 | (test) acc: 99.02%, loss: 0.032
[14/20] (train) acc: 99.7

## Deeper ... 

In [15]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(X, [-1, 28, 28, 1])

h1 = slim.conv2d(x_img, 32, kernel_size=[3,3]) # default relu, xavier
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
assert p1.shape[1:] == [14, 14, 32]

h2 = slim.conv2d(p1, 64, kernel_size=[3,3])
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
assert p2.shape[1:] == [7, 7, 64]

h3 = slim.conv2d(p2, 128, kernel_size=[3,3])
p3 = slim.max_pool2d(h3, kernel_size=[2,2], padding='same')
assert p3.shape[1:] == [4, 4, 128]

flat = slim.flatten(p3)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [16]:
train(epoch_n=100)

[ 1/100] (train) acc: 93.73%, loss: 0.204 | (test) acc: 98.47%, loss: 0.045
[ 2/100] (train) acc: 98.44%, loss: 0.051 | (test) acc: 98.67%, loss: 0.040
[ 3/100] (train) acc: 98.86%, loss: 0.036 | (test) acc: 98.83%, loss: 0.035
[ 4/100] (train) acc: 99.19%, loss: 0.027 | (test) acc: 98.59%, loss: 0.043
[ 5/100] (train) acc: 99.32%, loss: 0.022 | (test) acc: 99.09%, loss: 0.028
[ 6/100] (train) acc: 99.44%, loss: 0.018 | (test) acc: 99.18%, loss: 0.025
[ 7/100] (train) acc: 99.53%, loss: 0.015 | (test) acc: 99.01%, loss: 0.028
[ 8/100] (train) acc: 99.63%, loss: 0.011 | (test) acc: 99.19%, loss: 0.028
[ 9/100] (train) acc: 99.67%, loss: 0.011 | (test) acc: 99.20%, loss: 0.027
[10/100] (train) acc: 99.68%, loss: 0.009 | (test) acc: 99.00%, loss: 0.035
[11/100] (train) acc: 99.68%, loss: 0.010 | (test) acc: 98.98%, loss: 0.036
[12/100] (train) acc: 99.75%, loss: 0.007 | (test) acc: 99.27%, loss: 0.027
[13/100] (train) acc: 99.85%, loss: 0.004 | (test) acc: 99.24%, loss: 0.033
[14/100] (tr

## More Deeper ... (VGGNet-like)

In [17]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(X, [-1, 28, 28, 1])

h1 = slim.conv2d(x_img, 32, kernel_size=[3,3]) # default relu, xavier
h1 = slim.conv2d(h1, 32, kernel_size=[3,3])
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
assert p1.shape[1:] == [14, 14, 32]

h2 = slim.conv2d(p1, 64, kernel_size=[3,3])
h2 = slim.conv2d(h2, 64, kernel_size=[3,3])
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
assert p2.shape[1:] == [7, 7, 64]

h3 = slim.conv2d(p2, 128, kernel_size=[3,3])
h3 = slim.conv2d(h3, 128, kernel_size=[3,3])
p3 = slim.max_pool2d(h3, kernel_size=[2,2], padding='same')
assert p3.shape[1:] == [4, 4, 128]

flat = slim.flatten(p3)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [18]:
train(epoch_n=100)

[ 1/100] (train) acc: 94.92%, loss: 0.158 | (test) acc: 98.71%, loss: 0.040
[ 2/100] (train) acc: 98.70%, loss: 0.041 | (test) acc: 99.19%, loss: 0.024
[ 3/100] (train) acc: 99.10%, loss: 0.029 | (test) acc: 99.27%, loss: 0.021
[ 4/100] (train) acc: 99.34%, loss: 0.022 | (test) acc: 98.99%, loss: 0.029
[ 5/100] (train) acc: 99.37%, loss: 0.019 | (test) acc: 99.35%, loss: 0.022
[ 6/100] (train) acc: 99.53%, loss: 0.014 | (test) acc: 98.93%, loss: 0.032
[ 7/100] (train) acc: 99.56%, loss: 0.013 | (test) acc: 99.35%, loss: 0.022
[ 8/100] (train) acc: 99.61%, loss: 0.012 | (test) acc: 99.35%, loss: 0.024
[ 9/100] (train) acc: 99.68%, loss: 0.010 | (test) acc: 99.25%, loss: 0.029
[10/100] (train) acc: 99.70%, loss: 0.009 | (test) acc: 99.38%, loss: 0.020
[11/100] (train) acc: 99.75%, loss: 0.008 | (test) acc: 99.45%, loss: 0.025
[12/100] (train) acc: 99.75%, loss: 0.007 | (test) acc: 99.22%, loss: 0.033
[13/100] (train) acc: 99.74%, loss: 0.008 | (test) acc: 99.23%, loss: 0.030
[14/100] (tr

## dropout

In [19]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
training = tf.placeholder(tf.bool)

x_img = tf.reshape(X, [-1, 28, 28, 1])

h1 = slim.conv2d(x_img, 32, kernel_size=[3,3]) # default relu, xavier
h1 = slim.conv2d(h1, 32, kernel_size=[3,3])
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
d1 = slim.dropout(p1, keep_prob=0.7, is_training=training)
assert d1.shape[1:] == [14, 14, 32]

h2 = slim.conv2d(d1, 64, kernel_size=[3,3])
h2 = slim.conv2d(h2, 64, kernel_size=[3,3])
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
d2 = slim.dropout(p2, keep_prob=0.7, is_training=training)
assert d2.shape[1:] == [7, 7, 64]

h3 = slim.conv2d(d2, 128, kernel_size=[3,3])
h3 = slim.conv2d(h3, 128, kernel_size=[3,3])
p3 = slim.max_pool2d(h3, kernel_size=[2,2], padding='same')
d3 = slim.dropout(p3, keep_prob=0.7, is_training=training)
assert d3.shape[1:] == [4, 4, 128]

flat = slim.flatten(d3)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

### new `train()` with `training` placeholder

def train(epoch_n=20):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    batch_size = 100
    N = mnist.train.num_examples
    n_iter = N // batch_size

    for epoch in range(epoch_n):
        avg_loss = 0.
        avg_acc = 0.
        for _ in range(n_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, cur_acc, cur_loss = sess.run([train_op, accuracy, loss], 
                                            feed_dict={X: batch_x, y: batch_y, training: True})
            avg_acc += cur_acc
            avg_loss += cur_loss

        avg_acc /= n_iter
        avg_loss /= n_iter

        test_acc, test_loss = sess.run([accuracy, loss], 
                                       feed_dict={X: mnist.test.images, y: mnist.test.labels, training: False})
        print("[{:2}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
              format(epoch+1, epoch_n, avg_acc, avg_loss, test_acc, test_loss))

In [20]:
train(epoch_n=100, use_training_ph=True)

[ 1/100] (train) acc: 92.76%, loss: 0.224 | (test) acc: 98.65%, loss: 0.040
[ 2/100] (train) acc: 98.25%, loss: 0.060 | (test) acc: 99.01%, loss: 0.029
[ 3/100] (train) acc: 98.55%, loss: 0.046 | (test) acc: 98.96%, loss: 0.031
[ 4/100] (train) acc: 98.82%, loss: 0.037 | (test) acc: 99.32%, loss: 0.022
[ 5/100] (train) acc: 99.00%, loss: 0.032 | (test) acc: 99.33%, loss: 0.022
[ 6/100] (train) acc: 99.12%, loss: 0.028 | (test) acc: 99.34%, loss: 0.021
[ 7/100] (train) acc: 99.22%, loss: 0.024 | (test) acc: 99.32%, loss: 0.023
[ 8/100] (train) acc: 99.28%, loss: 0.023 | (test) acc: 99.27%, loss: 0.024
[ 9/100] (train) acc: 99.30%, loss: 0.022 | (test) acc: 99.46%, loss: 0.018
[10/100] (train) acc: 99.37%, loss: 0.019 | (test) acc: 99.50%, loss: 0.017
[11/100] (train) acc: 99.41%, loss: 0.018 | (test) acc: 99.52%, loss: 0.015
[12/100] (train) acc: 99.46%, loss: 0.017 | (test) acc: 99.34%, loss: 0.021
[13/100] (train) acc: 99.49%, loss: 0.016 | (test) acc: 99.52%, loss: 0.018
[14/100] (tr

## Batch normalization

In [21]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
training = tf.placeholder(tf.bool)

x_img = tf.reshape(X, [-1, 28, 28, 1])

bn_param = {'is_training': training, 'scale': True, 'decay': 0.99}
h1 = slim.conv2d(x_img, 32, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
h1 = slim.conv2d(h1, 32, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
p1 = slim.max_pool2d(h1, kernel_size=[2,2])
d1 = slim.dropout(p1, keep_prob=0.7, is_training=training)
assert d1.shape[1:] == [14, 14, 32]

h2 = slim.conv2d(d1, 64, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
h2 = slim.conv2d(h2, 64, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
p2 = slim.max_pool2d(h2, kernel_size=[2,2])
d2 = slim.dropout(p2, keep_prob=0.7, is_training=training)
assert d2.shape[1:] == [7, 7, 64]

h3 = slim.conv2d(d2, 128, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
h3 = slim.conv2d(h3, 128, kernel_size=[3,3], normalizer_fn=slim.batch_norm, normalizer_params=bn_param)
p3 = slim.max_pool2d(h3, kernel_size=[2,2], padding='same')
d3 = slim.dropout(p3, keep_prob=0.7, is_training=training)
assert d3.shape[1:] == [4, 4, 128]

flat = slim.flatten(d3)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)

# must do this even with slim
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [22]:
train(epoch_n=100, use_training_ph=True)

[ 1/100] (train) acc: 93.71%, loss: 0.201 | (test) acc: 97.05%, loss: 0.090
[ 2/100] (train) acc: 98.14%, loss: 0.059 | (test) acc: 98.47%, loss: 0.046
[ 3/100] (train) acc: 98.52%, loss: 0.049 | (test) acc: 98.99%, loss: 0.035
[ 4/100] (train) acc: 98.66%, loss: 0.043 | (test) acc: 99.35%, loss: 0.022
[ 5/100] (train) acc: 98.88%, loss: 0.035 | (test) acc: 99.24%, loss: 0.024
[ 6/100] (train) acc: 98.90%, loss: 0.035 | (test) acc: 99.02%, loss: 0.029
[ 7/100] (train) acc: 99.07%, loss: 0.029 | (test) acc: 98.94%, loss: 0.031
[ 8/100] (train) acc: 99.11%, loss: 0.027 | (test) acc: 99.29%, loss: 0.020
[ 9/100] (train) acc: 99.14%, loss: 0.027 | (test) acc: 99.16%, loss: 0.027
[10/100] (train) acc: 99.27%, loss: 0.024 | (test) acc: 99.27%, loss: 0.023
[11/100] (train) acc: 99.33%, loss: 0.021 | (test) acc: 99.20%, loss: 0.027
[12/100] (train) acc: 99.35%, loss: 0.020 | (test) acc: 99.32%, loss: 0.022
[13/100] (train) acc: 99.44%, loss: 0.018 | (test) acc: 99.15%, loss: 0.028
[14/100] (tr

# More slim by slim

In [23]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
training = tf.placeholder(tf.bool)

net = tf.reshape(X, [-1, 28, 28, 1])
n_filters = 32
bn_param = {'is_training': training, 'scale': True, 'decay': 0.99}
with slim.arg_scope([slim.conv2d], kernel_size=[3,3],
                    normalizer_fn=slim.batch_norm, normalizer_params=bn_param):
    for _ in range(3):
        net = slim.conv2d(net, n_filters)
        net = slim.conv2d(net, n_filters)
        net = slim.max_pool2d(net, kernel_size=[2,2], padding='same')
        net = slim.dropout(net, keep_prob=0.7, is_training=training)
        n_filters *= 2

flat = slim.flatten(net)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)

# must do this even with slim
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [24]:
train(epoch_n=100, use_training_ph=True)

[ 1/100] (train) acc: 93.34%, loss: 0.219 | (test) acc: 97.34%, loss: 0.081
[ 2/100] (train) acc: 98.04%, loss: 0.062 | (test) acc: 98.86%, loss: 0.034
[ 3/100] (train) acc: 98.49%, loss: 0.050 | (test) acc: 99.29%, loss: 0.023
[ 4/100] (train) acc: 98.66%, loss: 0.043 | (test) acc: 98.93%, loss: 0.035
[ 5/100] (train) acc: 98.83%, loss: 0.036 | (test) acc: 98.58%, loss: 0.044
[ 6/100] (train) acc: 99.01%, loss: 0.031 | (test) acc: 99.32%, loss: 0.023
[ 7/100] (train) acc: 99.03%, loss: 0.029 | (test) acc: 99.37%, loss: 0.021
[ 8/100] (train) acc: 99.16%, loss: 0.027 | (test) acc: 99.15%, loss: 0.025
[ 9/100] (train) acc: 99.15%, loss: 0.027 | (test) acc: 99.38%, loss: 0.018
[10/100] (train) acc: 99.26%, loss: 0.023 | (test) acc: 99.23%, loss: 0.024
[11/100] (train) acc: 99.33%, loss: 0.021 | (test) acc: 99.28%, loss: 0.025
[12/100] (train) acc: 99.34%, loss: 0.020 | (test) acc: 99.51%, loss: 0.017
[13/100] (train) acc: 99.37%, loss: 0.021 | (test) acc: 99.28%, loss: 0.024
[14/100] (tr

# Highest acc

In [36]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
training = tf.placeholder(tf.bool)

net = tf.reshape(X, [-1, 28, 28, 1])
n_filters = 64
bn_param = {'is_training': training, 'scale': True, 'decay': 0.999}
with slim.arg_scope([slim.conv2d], padding='same', normalizer_fn=slim.batch_norm, normalizer_params=bn_param):
    for i in range(3):
        net = slim.conv2d(net, n_filters, kernel_size=[3,3])
        net = slim.dropout(net, keep_prob=0.7, is_training=training)
        net = slim.conv2d(net, n_filters, kernel_size=[3,3])
        net = slim.dropout(net, keep_prob=0.7, is_training=training)
        if i == 2:
            net = slim.conv2d(net, n_filters, kernel_size=[1,1])
            net = slim.dropout(net, keep_prob=0.7, is_training=training)
        
        n_filters *= 2
        # 5x5 stride conv
        net = slim.conv2d(net, n_filters, kernel_size=[5,5], stride=2)
        net = slim.dropout(net, keep_prob=0.7, is_training=training)

flat = slim.flatten(net)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)

# must do this even with slim
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [37]:
train(epoch_n=200, use_training_ph=True)

[ 1/200] (train) acc: 89.56%, loss: 0.364 | (test) acc: 51.01%, loss: 3.299
[ 2/200] (train) acc: 97.69%, loss: 0.082 | (test) acc: 51.34%, loss: 4.660
[ 3/200] (train) acc: 98.04%, loss: 0.070 | (test) acc: 58.96%, loss: 2.740
[ 4/200] (train) acc: 98.42%, loss: 0.056 | (test) acc: 9.86%, loss: 18.193
[ 5/200] (train) acc: 98.51%, loss: 0.052 | (test) acc: 53.71%, loss: 2.925
[ 6/200] (train) acc: 98.67%, loss: 0.046 | (test) acc: 86.61%, loss: 0.618
[ 7/200] (train) acc: 98.92%, loss: 0.036 | (test) acc: 98.44%, loss: 0.073
[ 8/200] (train) acc: 98.97%, loss: 0.035 | (test) acc: 98.91%, loss: 0.057
[ 9/200] (train) acc: 99.17%, loss: 0.029 | (test) acc: 99.08%, loss: 0.042
[10/200] (train) acc: 99.13%, loss: 0.029 | (test) acc: 99.09%, loss: 0.054
[11/200] (train) acc: 99.15%, loss: 0.028 | (test) acc: 99.18%, loss: 0.046
[12/200] (train) acc: 99.26%, loss: 0.023 | (test) acc: 99.28%, loss: 0.035
[13/200] (train) acc: 99.33%, loss: 0.021 | (test) acc: 99.16%, loss: 0.038
[14/200] (tr

# Data augmentation

* Currently, our best model is not `highest acc`, but `BN`.
* Comparison with data augmentation

In [4]:
class AffineGenerator():
    def __init__(self, mnist):
        from keras.preprocessing.image import ImageDataGenerator
        
        self.mnist = mnist
        self.datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
        self.train_x = np.reshape(self.mnist.train.images, [-1, 28, 28, 1])
        self.train_y = self.mnist.train.labels

    def generate(self, batch_size=64):
        cnt = 0
        batch_n = self.train_x.shape[0] // batch_size
        for x, y in self.datagen.flow(self.train_x, self.train_y, batch_size=batch_size):
            ret_x = x.reshape(-1, 784)
            yield ret_x, y

            cnt += 1
            if cnt == batch_n:
                break

In [5]:
def train_da(epoch_n=20, batch_size=100, use_training_ph=False, datagen=AffineGenerator(mnist)):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    N = mnist.train.num_examples
    dq = collections.deque(maxlen=5)

    for epoch in range(epoch_n):
        avg_loss = 0.
        avg_acc = 0.

        n_iter = 0
        for batch_x, batch_y in datagen.generate(batch_size=batch_size):
            feed_dict = {X: batch_x, y: batch_y}
            if use_training_ph:
                feed_dict[training] = True
            _, cur_acc, cur_loss = sess.run([train_op, accuracy, loss], feed_dict=feed_dict)
            avg_acc += cur_acc
            avg_loss += cur_loss
            n_iter += 1

        avg_acc /= n_iter
        avg_loss /= n_iter

        test_acc = 0.
        test_loss = 0.
        for _ in range(mnist.test.num_examples // 1000):
            batch_x, batch_y = mnist.test.next_batch(1000)
            feed_dict = {X: batch_x, y: batch_y}
            if use_training_ph:
                feed_dict[training] = False
            cur_acc, cur_loss = sess.run([accuracy, loss], feed_dict=feed_dict)
            test_acc += cur_acc
            test_loss += cur_loss
        test_acc /= (mnist.test.num_examples // 1000)
        test_loss /= (mnist.test.num_examples // 1000)

        print("[{:2}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
              format(epoch+1, epoch_n, avg_acc, avg_loss, test_acc, test_loss))
        dq.append(test_acc)

    score = np.average(dq)
    print("average of last 5 test acc: {:.2%}".format(score))

Using TensorFlow backend.


In [51]:
# same as BN model

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
training = tf.placeholder(tf.bool)

net = tf.reshape(X, [-1, 28, 28, 1])
n_filters = 32
bn_param = {'is_training': training, 'scale': True, 'decay': 0.99}
with slim.arg_scope([slim.conv2d], kernel_size=[3,3],
                    normalizer_fn=slim.batch_norm, normalizer_params=bn_param):
    for _ in range(3):
        net = slim.conv2d(net, n_filters)
        net = slim.conv2d(net, n_filters)
        net = slim.max_pool2d(net, kernel_size=[2,2], padding='same')
        net = slim.dropout(net, keep_prob=0.7, is_training=training)
        n_filters *= 2

flat = slim.flatten(net)
logits = slim.fully_connected(flat, 10, activation_fn=None)
prob = tf.nn.softmax(logits)

correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss)

# must do this even with slim
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [52]:
train_da(epoch_n=100, use_training_ph=True)

[ 1/100] (train) acc: 88.60%, loss: 0.359 | (test) acc: 97.81%, loss: 0.066
[ 2/100] (train) acc: 96.40%, loss: 0.115 | (test) acc: 98.95%, loss: 0.031
[ 3/100] (train) acc: 97.18%, loss: 0.091 | (test) acc: 99.15%, loss: 0.026
[ 4/100] (train) acc: 97.73%, loss: 0.075 | (test) acc: 99.40%, loss: 0.018
[ 5/100] (train) acc: 97.89%, loss: 0.068 | (test) acc: 99.23%, loss: 0.024
[ 6/100] (train) acc: 97.98%, loss: 0.066 | (test) acc: 99.36%, loss: 0.017
[ 7/100] (train) acc: 98.29%, loss: 0.055 | (test) acc: 99.39%, loss: 0.018
[ 8/100] (train) acc: 98.37%, loss: 0.051 | (test) acc: 99.42%, loss: 0.020
[ 9/100] (train) acc: 98.38%, loss: 0.052 | (test) acc: 99.32%, loss: 0.022
[10/100] (train) acc: 98.57%, loss: 0.046 | (test) acc: 99.33%, loss: 0.022
[11/100] (train) acc: 98.57%, loss: 0.047 | (test) acc: 99.53%, loss: 0.014
[12/100] (train) acc: 98.61%, loss: 0.044 | (test) acc: 99.56%, loss: 0.015
[13/100] (train) acc: 98.64%, loss: 0.044 | (test) acc: 99.37%, loss: 0.018
[14/100] (tr

# Ensemble

In [37]:
def one_hot(dense, n_class=10):
    N = dense.shape[0]
    ret = np.zeros([N, n_class])
    ret[np.arange(N), dense] = 1
    return ret

In [22]:
class BestModel(object):
    def __init__(self, lr):
        X = tf.placeholder(tf.float32, [None, 784])
        y = tf.placeholder(tf.float32, [None, 10])
        training = tf.placeholder(tf.bool)

        net = tf.reshape(X, [-1, 28, 28, 1])
        n_filters = 32
        bn_param = {'is_training': training, 'scale': True, 'decay': 0.99}
        with slim.arg_scope([slim.conv2d], kernel_size=[3,3],
                            normalizer_fn=slim.batch_norm, normalizer_params=bn_param):
            for _ in range(3):
                net = slim.conv2d(net, n_filters)
                net = slim.conv2d(net, n_filters)
                net = slim.max_pool2d(net, kernel_size=[2,2], padding='same')
                net = slim.dropout(net, keep_prob=0.7, is_training=training)
                n_filters *= 2

        flat = slim.flatten(net)
        logits = slim.fully_connected(flat, 10, activation_fn=None)
        prob = tf.nn.softmax(logits)

        # add predict ops for majority voting ensemble
        predict = tf.argmax(logits, axis=1)
        correct = tf.equal(predict, tf.argmax(y, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
        loss = tf.reduce_mean(loss)

        # must do this even with slim
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
        
        # for interaction
        self.X = X
        self.y = y
        self.training = training
        self.predict = predict
        self.accuracy = accuracy
        self.loss = loss
        self.train_op = train_op

In [53]:
N = mnist.train.num_examples
dq = collections.deque()
datagen=AffineGenerator(mnist)

lrs = [0.0003, 0.0007, 0.001, 0.002, 0.003]
configs = [
    {'lr': 0.0003, 'batch_size':  50, 'epoch_n':  80},
    {'lr': 0.0007, 'batch_size': 100, 'epoch_n': 100},
    {'lr':  0.001, 'batch_size': 100, 'epoch_n': 100},
    {'lr':  0.002, 'batch_size': 200, 'epoch_n': 120},
    {'lr':  0.003, 'batch_size': 300, 'epoch_n': 150}
]

pred_vote = np.zeros([mnist.test.num_examples, 10])
for i, cfg in enumerate(configs):
    tf.reset_default_graph()
    model = BestModel(cfg['lr'])
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print("Train model{} ... {}".format(i, cfg))
        
        for epoch in range(cfg['epoch_n']):
            avg_loss = 0.
            avg_acc = 0.

            n_iter = 0
            for batch_x, batch_y in datagen.generate(batch_size=cfg['batch_size']):
                feed_dict = {model.X: batch_x, model.y: batch_y, model.training: True}
                _, cur_acc, cur_loss = sess.run([model.train_op, model.accuracy, model.loss], feed_dict=feed_dict)
                avg_acc += cur_acc
                avg_loss += cur_loss
                n_iter += 1

            avg_acc /= n_iter
            avg_loss /= n_iter

            feed_dict = {model.X: mnist.test.images, model.y: mnist.test.labels, model.training: False}
            test_acc, test_loss = sess.run([model.accuracy, model.loss], feed_dict=feed_dict)
            
            print("[{:2}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
                  format(epoch+1, cfg['epoch_n'], avg_acc, avg_loss, test_acc, test_loss))
    
        # use last model for ensemble
        feed_dict = {model.X: mnist.test.images, model.y: mnist.test.labels, model.training: False}
        test_acc, test_pred = sess.run([model.accuracy, model.predict], feed_dict=feed_dict)
        dq.append(test_acc)
        pred_vote += one_hot(test_pred)
        print

print("last test accs for each model")
for i, acc in enumerate(dq):
    print("model{}: {:.2%}".format(i, acc))

ensemble_acc = np.average(np.argmax(mnist.test.labels, axis=1) == np.argmax(pred_vote, axis=1))
print("ensemble: {:.2%}".format(ensemble_acc))

Train model0 ... {'lr': 0.0003, 'batch_size': 50, 'epoch_n': 80}
[ 1/80] (train) acc: 85.35%, loss: 0.453 | (test) acc: 98.43%, loss: 0.046
[ 2/80] (train) acc: 95.77%, loss: 0.136 | (test) acc: 98.97%, loss: 0.032
[ 3/80] (train) acc: 96.84%, loss: 0.104 | (test) acc: 99.24%, loss: 0.024
[ 4/80] (train) acc: 97.27%, loss: 0.088 | (test) acc: 99.13%, loss: 0.027
[ 5/80] (train) acc: 97.65%, loss: 0.074 | (test) acc: 99.31%, loss: 0.021
[ 6/80] (train) acc: 97.83%, loss: 0.070 | (test) acc: 99.42%, loss: 0.018
[ 7/80] (train) acc: 98.03%, loss: 0.062 | (test) acc: 99.27%, loss: 0.022
[ 8/80] (train) acc: 98.20%, loss: 0.057 | (test) acc: 99.36%, loss: 0.022
[ 9/80] (train) acc: 98.31%, loss: 0.056 | (test) acc: 99.45%, loss: 0.018
[10/80] (train) acc: 98.37%, loss: 0.052 | (test) acc: 99.42%, loss: 0.017
[11/80] (train) acc: 98.51%, loss: 0.048 | (test) acc: 99.50%, loss: 0.016
[12/80] (train) acc: 98.54%, loss: 0.047 | (test) acc: 99.42%, loss: 0.018
[13/80] (train) acc: 98.64%, loss: 

IndexError: tuple index out of range

In [56]:
print("last test accs for each model")
for i, acc in enumerate(dq):
    print("model{}: {:.2%}".format(i, acc))

ensemble_acc = np.average(np.argmax(mnist.test.labels, axis=1) == np.argmax(pred_vote, axis=1))
print("ensemble: {:.2%}".format(ensemble_acc))

last test accs for each model
model0: 99.62%
model1: 99.68%
model2: 99.71%
model3: 99.63%
model4: 99.58%
ensemble: 99.71%


# Very-Advanced: Snap-shot ensemble